In [1]:
####################### Create a Customer Travel Destinations Map #######################

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rohnert Park,US,38.3396,-122.7011,53.01,81,1,0.60,clear sky
1,1,Rikitea,PF,-23.1203,-134.9692,77.38,72,96,12.06,overcast clouds
2,2,Coolum Beach,AU,-26.5333,153.1000,77.00,97,96,10.69,overcast clouds
3,3,Albany,US,42.6001,-73.9662,28.99,75,100,4.45,snow
4,4,Jamestown,US,42.0970,-79.2353,19.99,73,90,17.27,light snow


In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature? "))
max_temp = float(input("What is the maximum temperature? "))

What is the minimum temperature? 75
What is the maximum temperature? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Rikitea,PF,-23.1203,-134.9692,77.38,72,96,12.06,overcast clouds
2,2,Coolum Beach,AU,-26.5333,153.1000,77.00,97,96,10.69,overcast clouds
6,6,Damaturu,NG,11.7470,11.9608,80.58,17,49,8.52,scattered clouds
15,15,Cidreira,BR,-30.1811,-50.2056,76.41,62,35,10.42,scattered clouds
17,17,Tomatlan,MX,19.9333,-105.2500,87.69,44,13,4.21,few clouds
19,19,Puerto Ayora,EC,-0.7393,-90.3518,84.20,70,0,17.27,light rain
28,28,Butaritari,KI,3.0707,172.7902,80.64,81,43,18.21,scattered clouds
29,29,Mahebourg,MU,-20.4081,57.7000,80.60,74,20,10.36,few clouds
31,31,Kuching,MY,1.5500,110.3333,75.20,100,20,3.44,few clouds
35,35,Keita,NE,14.7551,5.7749,89.60,8,41,4.61,scattered clouds


In [9]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                203
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [10]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna() 

In [11]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,77.38,overcast clouds,-23.1203,-134.9692,
2,Coolum Beach,AU,77.00,overcast clouds,-26.5333,153.1000,
6,Damaturu,NG,80.58,scattered clouds,11.7470,11.9608,
15,Cidreira,BR,76.41,scattered clouds,-30.1811,-50.2056,
17,Tomatlan,MX,87.69,few clouds,19.9333,-105.2500,
19,Puerto Ayora,EC,84.20,light rain,-0.7393,-90.3518,
28,Butaritari,KI,80.64,scattered clouds,3.0707,172.7902,
29,Mahebourg,MU,80.60,few clouds,-20.4081,57.7000,
31,Kuching,MY,75.20,few clouds,1.5500,110.3333,
35,Keita,NE,89.60,scattered clouds,14.7551,5.7749,


In [13]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    #Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
# Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,77.38,overcast clouds,-23.1203,-134.9692,Pension Maro'i
2,Coolum Beach,AU,77.00,overcast clouds,-26.5333,153.1000,Palmer Coolum Resort
6,Damaturu,NG,80.58,scattered clouds,11.7470,11.9608,"Hayas Guest Inn, Central Roundabout, Gujba Road"
15,Cidreira,BR,76.41,scattered clouds,-30.1811,-50.2056,Hotel Castelo
17,Tomatlan,MX,87.69,few clouds,19.9333,-105.2500,Norma franco


In [17]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Rikitea,PF,77.38,overcast clouds,-23.1203,-134.9692,Pension Maro'i
2,Coolum Beach,AU,77.00,overcast clouds,-26.5333,153.1000,Palmer Coolum Resort
6,Damaturu,NG,80.58,scattered clouds,11.7470,11.9608,"Hayas Guest Inn, Central Roundabout, Gujba Road"
15,Cidreira,BR,76.41,scattered clouds,-30.1811,-50.2056,Hotel Castelo
17,Tomatlan,MX,87.69,few clouds,19.9333,-105.2500,Norma franco


In [19]:
# Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# Rename the column "Current Description" to "Current Weather".
clean_hotel_df.columns = ["City","Country","Max Temp","Current Weather","Lat","Lng","Hotel Name"]
clean_hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
1,Rikitea,PF,77.38,overcast clouds,-23.1203,-134.9692,Pension Maro'i
2,Coolum Beach,AU,77.00,overcast clouds,-26.5333,153.1000,Palmer Coolum Resort
6,Damaturu,NG,80.58,scattered clouds,11.7470,11.9608,"Hayas Guest Inn, Central Roundabout, Gujba Road"
15,Cidreira,BR,76.41,scattered clouds,-30.1811,-50.2056,Hotel Castelo
17,Tomatlan,MX,87.69,few clouds,19.9333,-105.2500,Norma franco


In [21]:
# Using the template add hotel name, the city name, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F</dd>  
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))